## Convo Summary

In this notebook, we loook at different types of LLMS and how they summarize audio transcripts. Specifically we are interested in:
- How concise each model summarizes the transcripts to
- How understandable the summaries are- measured in terms of how quickly a receiver may view some measurement of "urgency"

In [11]:
#imports
import os
import openai
from transformers import pipeline
from dotenv import load_dotenv

We store a list of prompts to reuse for every model.

In [12]:
concise_prompt = "Summarize the following audio transcript in 3–4 sentences. Focus on the main issue discussed, the speaker’s emotional state, and any signs of urgency or distress"
analytical_prompt = "Read this conversation transcript and provide a concise summary that identifies the key concern, emotional tone, and any notable escalation or risk indicators. Use professional and neutral language."
structured_prompt = "Summarize the transcript with the following format:\n- Main issue: \n- Emotional state: \n- Urgency level (low/medium/high):"

list_of_prompts = [concise_prompt, analytical_prompt, structured_prompt]

## ChatGPT

We start by looking into summaries generated by ChatGPT 4o.

In [ ]:
# Setting up the API Key, stored in seperate environment file
load_dotenv()  # Loads from .env by default
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# Function to prompt ChatGPT
def get_chatgpt_response(prompt):
    completion = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content


def store_chatgpt_responses(list_of_prompts):
    # Loop through each prompt
    for prompt_num, prompt in enumerate(list_of_prompts, start=1):
        # Define output path for this prompt's results
        output_filename = f"Summaries/gpt-4/prompt{prompt_num}_summary.txt"
        
        # Create/open the file for writing all call summaries under this prompt
        with open(output_filename, "w", encoding="utf-8") as out_file:
            # Loop through all transcripts (call01–call05)
            for i in range(1, 6):
                filename = f"call{i:02d}.txt"
                filepath = f"Transcripts/{filename}"  # based on your folder structure

                with open(filepath, "r", encoding="utf-8") as f:
                    transcript_text = f.read()

                # Combine prompt with transcript
                full_prompt = f"{prompt}\n\nTranscript:\n{transcript_text}"
                response = get_chatgpt_response(full_prompt)

                # Write formatted summary to the output file
                out_file.write(f"=== Summary for {filename} ===\n")
                out_file.write(response.strip() + "\n\n")  # add space between summaries

            print(f"Saved all summaries for prompt {prompt_num} to {output_filename}")


Calling the function

In [ ]:
store_chatgpt_responses(list_of_prompts)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In each txt file, we are able to see how each prompt summarizes the five different audio transcripts. We observe that the **structured_prompt** does the best in summarizing in **the most organized/easily understandable manner**.

## BART

We also wanted to try summarizing with BART, a model pre-trained on English language and fine-tuned on CNN Daily Mail. 

In [19]:
# Load BART summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to generate summary using BART
def get_bart_summary(text, max_input_length=1024, max_output_length=200):
    # Truncate input if it exceeds BART's limit (1024 tokens)
    truncated_text = text[:max_input_length]

    summary = summarizer(
        truncated_text,
        max_length=max_output_length,
        min_length=30,
        do_sample=False
    )
    return summary[0]['summary_text']


# Function to store summaries using BART
def store_bart_summaries(list_of_prompts):
    os.makedirs("Summaries/bart", exist_ok=True)

    for prompt_num, prompt in enumerate(list_of_prompts, start=1):
        output_filename = f"Summaries/bart/prompt{prompt_num}_summary.txt"

        with open(output_filename, "w", encoding="utf-8") as out_file:
            for i in range(1, 6):
                filename = f"call{i:02d}.txt"
                filepath = f"Transcripts/{filename}"

                with open(filepath, "r", encoding="utf-8") as f:
                    transcript_text = f.read()

                # Combine prompt with transcript
                full_input = f"{prompt}\n\nTranscript:\n{transcript_text}"

                # Generate summary using BART
                summary = get_bart_summary(full_input)

                # Write the summary to the output file
                out_file.write(f"=== Summary for {filename} ===\n")
                out_file.write(summary.strip() + "\n\n")

            print(f"Saved all summaries for prompt {prompt_num} to {output_filename}")


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

c:\Users\emily\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\emily\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [20]:
store_bart_summaries(list_of_prompts)

Saved all summaries for prompt 1 to Summaries/bart/prompt1_summary.txt
Saved all summaries for prompt 2 to Summaries/bart/prompt2_summary.txt
Saved all summaries for prompt 3 to Summaries/bart/prompt3_summary.txt


From the summaries produced by bart, it's clear that the model attempts to summarize more like a new article summary than an audio transcript summarization. Bart is not a suitable model for summarization.